Lê Nguyễn Minh Huy - 20521394

## Part 1. Explore interesting properties of the word vectors with GenSim 


### Import gensim

In [2]:
import gensim

### Download the pretrained model

In [3]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
# Unzip the model
!unzip wiki-news-300d-1M.vec.zip

--2022-11-15 15:48:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  30.2MB/s    in 19s     

2022-11-15 15:49:04 (33.5 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [4]:
# Load the model
model = gensim.models.KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec', binary=False)

Let's try the very first step: trying to represent a token by a feature vector.

For example, we use the following syntax to map the word `'king'` to a 300-dimensonal vector:

In [5]:
king = model['king']
print(king)

[ 1.082e-01  4.450e-02 -3.840e-02  1.100e-03 -8.880e-02  7.130e-02
 -6.960e-02 -4.770e-02  7.100e-03 -4.080e-02 -7.070e-02 -2.660e-02
  5.000e-02 -8.240e-02  8.480e-02 -1.627e-01 -8.510e-02 -2.950e-02
  1.534e-01 -1.828e-01 -2.208e-01  2.430e-02 -9.210e-02 -1.089e-01
 -1.009e-01 -1.190e-02  3.770e-02  2.038e-01  7.200e-02  2.020e-02
  2.798e-01  1.150e-02 -1.510e-02  1.037e-01  4.000e-04 -1.040e-02
  1.960e-02  1.265e-01  8.280e-02 -1.369e-01  1.070e-01  1.270e-01
 -3.490e-02 -6.830e-02 -1.140e-02  3.370e-02  1.260e-02  7.920e-02
  4.400e-02 -2.530e-02  4.890e-02 -7.850e-02 -6.259e-01 -9.720e-02
  1.654e-01 -5.780e-02 -4.370e-02  4.090e-02 -1.820e-02 -1.891e-01
  2.770e-02 -1.460e-02 -5.310e-02  4.260e-02  4.900e-03  4.000e-03
  1.423e-01 -9.750e-02 -3.500e-03  9.630e-02 -1.900e-03 -1.466e-01
 -1.662e-01  6.650e-02 -1.500e-01 -1.267e-01  2.670e-02 -1.560e-01
 -1.442e-01  1.515e-01  2.420e-02 -6.080e-02  9.180e-02 -2.407e-01
 -4.110e-02 -1.420e-02  6.550e-02 -3.590e-02  1.459e-01  9.400

We can use that feature vector compute the similarity with other words.

In [6]:
car = model['car']
queen = model['queen']
power = model['power']

print('queen and king: ', queen.dot(king))
print('power and king: ', power.dot(king))
print('car and king: ', car.dot(king))

queen and king:  3.2907822
power and king:  1.6147376
car and king:  1.4670291


In [7]:
like = model['like']
love = model['love']
hate = model['hate']

print('love and like: ', love.dot(like))
print('like and hate: ', like.dot(hate))
print('love and hate: ', love.dot(hate))

love and like:  1.6091172
like and hate:  1.8135115
love and hate:  2.8157127


### Linguistic regularities

In [8]:
# king - man + woman = queen
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

[('queen', 0.7515910863876343), ('monarch', 0.6741327047348022), ('princess', 0.6713887453079224), ('kings', 0.6698989868164062), ('kingdom', 0.5971318483352661), ('royal', 0.5921063423156738), ('uncrowned', 0.5911505818367004), ('prince', 0.5909028053283691), ('lady', 0.5904011130332947), ('monarchs', 0.5884358286857605)]


In [9]:
# queen + man - king = woman
result = model.most_similar(positive=['queen', 'man'], negative=['king'])
print("If the king is a man than the queen is:", result[0][0])

If the king is a man than the queen is: woman


### Another relations

## Part 2. Word2vec with Tensorflow

In [10]:
# capital - country
capital = model.most_similar(["Paris", "Britain"], ["France"], topn=1)
print("If Paris is capital of France, capital of Britain is:", capital)

Capital of Britain is: [('London', 0.7543942928314209)]


In [17]:
# noun - plural noun
plural = model.most_similar(["pens", "apple"], ["pen"], topn=1)
print("If pens is plural noun of pen, plural noun of apple is:", plural)

If pens is plural noun of pen, plural noun of apple is: [('apples', 0.7407915592193604)]


In [20]:
# verb - v3 form
v3 = model.most_similar(["attended", "play"], ["attend"], topn=1)
print("If attended is v3 of attend, v3 of play is:", v3)

If attended is v3 of attend, v3 of play is: [('played', 0.7761051654815674)]


### import model

In [13]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/Wiki-words-250-with-normalization/2")


In [14]:
love_vec = embed(["love"]).numpy()
like_vec = embed(["like"]).numpy()
hate_vec = embed(["hate"]).numpy()

In [15]:
love_vec.dot(like_vec.T)

array([[0.24091528]], dtype=float32)

In [16]:
love_vec.dot(hate_vec.T)

array([[0.5915959]], dtype=float32)